# **ADSP - P8 PROJECT** - scientific papers classifier based on NLP
---
**CANDIDATES**

GILLIO FRANCESCO

SEMINARIO YRIGOYEN CESAR AUGUSTO

**COURSE** Applied Data Science Project

**University** Politecnico di Torino - Italy


## Project description

**BertTopic** is used to create a semi-supervised model with labels available

**Dataset** can be downloaded fro hugginface at the link: [Wiki Medical Terms](https://huggingface.co/datasets/gamino/wiki_medical_terms)

 ## <span style="font-family:Verdana; color:blue">  0. TO DO

<span style="font-family:Verdana; color:blue">

- download similar dataset from hugging-face to start practicing
- data cleaning
- create embeddings
- export embeddings
- start with the model
***

## 1. Exploring the dataset

In [4]:
%%capture
!pip install bertopic
!pip install datasets
!pip install openai

In [5]:
from datasets import load_dataset
import tqdm as notebook_tqdm

dataset = load_dataset("dhiruHF/research_paper_multi_label_data_balanced")
terms_number= len(dataset['train'])
print(f"N° wiki terms: {terms_number}")

N° wiki terms: 1985


In [38]:
def get_chapters(text):
    chapters = text.split('###')[2]
    labels = text.split('###')[3]
    labels = labels.split(':')[1]
    title=chapters.split('\n')[2].split(':')[1]
    abstract=chapters.split('\n')[3:]
    abstract[0]=abstract[0][9:]
    delimiter="\n"
    abstract=delimiter.join(abstract)
    #labels=labels.split(',')
    #labels=[label.strip() for label in labels]
    item_dict={}
    item_dict['title']=title
    item_dict['abstract']=abstract
    item_dict['labels']=labels
    return item_dict


In [41]:
item=get_chapters(dataset['train'][0]['text'])
print(item['title'])

 Reconstructing Subject-Specific Effect Maps


In [26]:
dataset['train'][0]['text'].split('###')[2]

"RESEARCH PAPER:\n\nTitle: Reconstructing Subject-Specific Effect Maps\nAbstract:   Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we propose a reconstruction\nmethod, named RSM, to improve subject-specific detections of predictive\nmodeling approaches and in particular, binary classifiers. RSM specifically\naims to reduce noise due to sampling error associated with using a finite\nsample of examples to train classifiers. The proposed method is a wrap

In [42]:
data_dict=[get_chapters(item) for item in dataset['train']['text']]

In [46]:
print(f"title:\n {data_dict[34]['title']}")
print(f"abstract:\n {data_dict[34]['abstract']}")
print(f"labels:\n {data_dict[34]['labels']}")

title:
  Deep Neural Network Optimized to Resistive Memory with Nonlinear Current-Voltage Characteristics
abstract:
    Artificial Neural Network computation relies on intensive vector-matrix
multiplications. Recently, the emerging nonvolatile memory (NVM) crossbar array
showed a feasibility of implementing such operations with high energy
efficiency, thus there are many works on efficiently utilizing emerging NVM
crossbar array as analog vector-matrix multiplier. However, its nonlinear I-V
characteristics restrain critical design parameters, such as the read voltage
and weight range, resulting in substantial accuracy loss. In this paper,
instead of optimizing hardware parameters to a given neural network, we propose
a methodology of reconstructing a neural network itself optimized to resistive
memory crossbar arrays. To verify the validity of the proposed method, we
simulated various neural network with MNIST and CIFAR-10 dataset using two
different specific Resistive Random Access Me

## 2. Preparing the dataset for BERTopic architecture

In [ ]:
# functions preparing the dataset for BERTopic architecture


In [ ]:
#CLEANING DUPLICATES & NAN SECTION VALUES

print(f"initial size: {md_terms_df.shape}")
# removing duplicates
md_terms_ss_df=md_terms_df.drop_duplicates(subset=['medical_term'], keep='first')
print(f"new size of items after removing duplicates: {md_terms_ss_df.shape}")

#removing nan or empty sign & symptomps terms
md_terms_ss_df=md_terms_ss_df[md_terms_ss_df['signs_and_symptoms'].astype(bool)]
# md_terms_df=md_terms_df.dropna( )
print(f"new size of items after removing na or empty sign & symptomps: {md_terms_ss_df.shape}")
md_terms_ss_df.to_excel('diseases_correct.xlsx')
# md_terms_ss_df=add_groundtruth("ICD-11_class.xlsx",md_terms_ss_df)

initial size: (6861, 3)
new size of items after removing duplicates: (6762, 3)
new size of items after removing na or empty sign & symptomps: (2255, 3)


In [ ]:
''' wordcloud example
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = md_terms_ss_df['description'][52]

x, y = np.ogrid[:300, :300]

mask = (x - 150) ** 2 + (y - 150) ** 2 > 130 ** 2
mask = 255 * mask.astype(int)


wc = WordCloud(background_color="white", repeat=True, mask=mask)
wc.generate(text)

plt.axis("off")
plt.imshow(wc, interpolation="bilinear")
plt.show()
'''

In [ ]:
#PREPARING DATASET FOT BERTOPIC
import json
data_dict=[]
for idx in md_terms_ss_df.index:
    tmp_dict={}
    tmp_dict['idx']=idx
    tmp_dict['md_term']=md_terms_ss_df['medical_term'][idx]
    tmp_dict['md_description']=md_terms_ss_df['description'][idx]
    tmp_dict['Sign and Symptoms']=create_paragraph(md_terms_ss_df['signs_and_symptoms'][idx])
    data_dict.append(tmp_dict)

print("dictionary terms (useful to be classificated)")
print(json.dumps(data_dict[50],indent=4))
#print(wiki_description[md_terms_ss_df['medical_term'][50]])
# s2=create_paragraph(possible_signs_syntomps[1])
# print(possible_signs_syntomps[1])
# print(f"as paragraph: {s2}")

dictionary terms (useful to be classificated)
{
    "idx": 140,
    "md_term": "Type 1 diabetes",
    "md_description": "Type 1 diabetes (T1D), formerly known as juvenile diabetes, is an autoimmune disease that originates when cells that make insulin (beta cells) are destroyed by the immune system. Insulin is a hormone required for the cells to use blood sugar for energy and it helps regulate glucose levels in the bloodstream. Before treatment this results in high blood sugar levels in the body. The common symptoms of this elevated blood sugar are frequent urination, increased thirst, increased hunger, weight loss, and other serious complications. Additional symptoms may include blurry vision, tiredness, and slow wound healing. Symptoms typically develop over a short period of time, often a matter of weeks.The cause of type 1 diabetes is unknown, but it is believed to involve a combination of genetic and environmental factors. The underlying mechanism involves an autoimmune destruction

In [ ]:
# disease_terms=[]
# disease_descriptions=[]
# disease_sign_syntomps=[]
# i=0
# for syntomp in signs_symptoms:
#     if(syntomp!=[]):
#         disease_terms.append(wiki_terms[i])
#         disease_descriptions.append(wiki_description[i].split('\n')[0] )
#         disease_sign_syntomps.append(syntomp)
#         i+=1
# data_dict_2={'medical_term':disease_terms,'description':disease_descriptions,'signs_and_symptoms':disease_sign_syntomps}
# diseases_df=pd.DataFrame(data_dict_2)
# print(diseases_df.head(2))
# print(f"{len(disease_sign_syntomps)} diseases found in the dataset")
# diseases_df.to_excel('diseases.xlsx')
# syntomps_as_paragrapgh=[create_paragraph(s) for s in disease_sign_syntomps]
symtomps_as_paragraph=[]
symtomps_as_paragraph=[create_paragraph(s) for s in md_terms_ss_df['signs_and_symptoms']]
print(len(symtomps_as_paragraph))


2255


## 3. Setting up base BERTopic model

### 3.1 Precalculate embeddings


In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
embeddings = embedding_model.encode(symtomps_as_paragraph, show_progress_bar=True)

Batches: 100%|██████████| 71/71 [05:55<00:00,  5.01s/it]


### 3.2 Setting up bertopic blocks

In [ ]:
#!pip install 'umap-learn==0.3.10'
from umap import UMAP
umap_model = UMAP(n_neighbors=20, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
additional_stopwords=['Signs','signs','Sign', 'sign', 'Symptoms','symptoms','Symptom','symptom','include','affected','people','features','patients','disease','present','common','usually'] #'syndrome', disease, patient, patients
stop_words_mod=list(text.ENGLISH_STOP_WORDS)+additional_stopwords
vectorizer_model = CountVectorizer(stop_words=stop_words_mod, min_df=1, ngram_range=(1, 2))
# vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

### 3.3 DEFINING representation models

In [ ]:
#!pip install typing-extensions --upgrade

import openai
import bertopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech
#from spacy import parts_of_speech
# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
#pos_model = PartOfSpeech("en_core_web_sm")
# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI(api_key="sk-...")
#openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    #"POS": pos_model
}

## 4. Model exploration

### Tests:
HDBSCAN - choose reasonable number of topics


Vectorizer tests
- mindf=1 : changes just group 12: war--- to veteran.sle, war, gulf. # of topics are the same. SLe is the most representative doc, it's autoimmune disease


In [ ]:
from bertopic import BERTopic
from sklearn.cluster import KMeans
#put here the block to study in order to understand behaviors
# list_to_check=range(20,160,10)
list_to_check=[(1,3)]

params=[]
# list_to_check=range(140,160,10)
for size in list_to_check:#reversed(list_to_check):
  # hdbscan_model = HDBSCAN(min_cluster_size=size, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
  vectorizer_model = CountVectorizer(stop_words=stop_words_mod, min_df=1, ngram_range=size)
  # hdbscan_model= KMeans(n_clusters=size)
  topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)
  topics, probs = topic_model.fit_transform(symtomps_as_paragraph, embeddings)
  print(f"size {size} analyzed")
  a=topic_model.get_topic_freq()
  # a['index_old']=a.index
  b=topic_model.get_topic_info()['Name']
  a.reset_index(inplace=True)
  b.reset_index(drop=True)
  a=pd.concat([a,b],axis=1)
  params.append(a)
  topic_model.get_topic_info()

2024-02-18 22:59:19,437 - BERTopic - Reduced dimensionality
2024-02-18 22:59:19,504 - BERTopic - Clustered reduced embeddings


size (1, 3) analyzed


In [ ]:
topic_model.get_topic_info()
# params[13].head()

In [ ]:
topic_model.get_representative_docs(15)

### saving model

In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("./model_dir/hdbscan", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

### Accuracy Calculation
- find metrics - remember ICD11 per clustering

#### OPEC evaluation

In [ ]:
#creating dataset ready for opec



### Creating plots

In [ ]:
%matplotlib inline


### Model visualization: topics and documents

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
# Documents
from umap import UMAP
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
labels=md_terms_ss_df['medical_term'].to_list()
topic_model.visualize_documents(labels, reduced_embeddings=reduced_embeddings)